# User-based Collaborative Filtering (CF)
The goal is to find "users like me".

1. [User-User CF](#useruser)
2. [Item-Item CF](#itemitem)

In [ ]:
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime
from sortedcontainers import SortedList

In [ ]:
# load in the data
import os 
with open('user2movie.json', 'rb') as f:
    user2movie = pickle.load(f)

with open('movie2user.json', 'rb') as f:
    movie2user = pickle.load(f)

with open('usermovie2rating.json', 'rb') as f:
    usermovie2rating = pickle.load(f)

with open('usermovie2rating_test.json', 'rb') as f:
    usermovie2rating_test = pickle.load(f)


## <a id='useruser'>1. User-User CF </a>
Check out the **userbased.py** for clean code.

In [ ]:
N = np.max(list(user2movie.keys()))+1
# The test set may contain movises the train set does not have data on
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2) + 1
print("N:", N, "M:", M)

#### All the stuff that need to be calculated:
1. Each users average rating (r_bar)
2. For each user i, we need to know its neighbors i'
3. So I'm going to use these neighbors who already rated movie j to get a prediction for what user I would rate movie j (calculate all the weights and then sort -> n^2 calculation)
4. It's actually useful to store the deviations as well.

<img src="images/score.png" alt="score" width="300" height="200">
(user i, movie j)

WE use Pearson correlation to estimate the similarity. The Pearson correlation coefficient ($r$) is a measure of the linear relationship between two variables $X$ and $Y$. It is calculated using the following formula:

$$
r = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2 \sum_{i=1}^{n} (y_i - \bar{y})^2}}
$$



In [ ]:
# to find the user similarities, you have to do O(N^2 * M) calculations!
# in the "real-world" you'd want to parallelize this
# note: we really only have to do half the calculations, since w_ij is symmetric
K = 25 # number of neighbors we'd like to consider
limit = 5 # number of common movies users must have in common in order to consider
neighbors = [] # store each user's neighbors in this list
averages = [] # each user's average rating
deviations = [] # each user's deviation list

# find the K closest users to user i
for i in range(N):
    movies_i = user2movie[i]
    movies_i_set = set(movies_i)
    # calcualte user i's avg and dev
    ratings_i = {movie: usermovie2rating[(i, movie)] for movie in movies_i}
    avg_i = np.mean(list(ratings_i.values()))
    dev_i = {movie: (avg_i - usermovie2rating[(i, movie)]) for movie in movies_i}
    dev_i_vals = np.array(list(dev_i.values()))
    sigma_i = np.sqrt(dev_i_vals.dot(dev_i_vals)) # denominator in PC
    
    averages.append(avg_i)
    deviations.append(dev_i)
    
    # loop through all the other N-1 users
    sl = SortedList()
    for j in range(N):
        if j != i:
            movies_j = user2movie[j]
            movies_j_set = set(movies_j)
            common_movies = (movies_i_set & movies_j_set)
            if len(common_movies) > limit:
                # calc avg and dev
                ratings_j = {movie: usermovie2rating[(j, movie)] for movie in movies_j}
                avg_j = np.mean(list(ratings_j.values()))
                dev_j = {m: (avg_j - usermovie2rating[(j, m)]) for m in movies_j}
                dev_j_vals = np.array(list(dev_j.values()))
                sigma_j = np.sqrt(dev_i_vals.dot(dev_i_vals))
                
                # calc correlation coefficient
                numerator = sum(dev_i[m] * dev_j[m] for m in common_movies)
                denominator = sigma_i * sigma_j
                wij = numerator / denominator
                
                sl.add((-wij, j)) # negate weight becasue list is sorted ascending
                if len(sl) > K:
                    del sl[-1]
    
    # store neighbors
    neighbors.append(sl)
    
    if i % 1 == 0:
        print(i)

Use neighbors, calc train and test MSE

In [ ]:
# i for current user, i_prime for all the other user,  m for movie
def predict(i, m): 
    numerator = 0
    denominator = 0
    for neg_w, i_prime in neighbors[i]:
        try:
            numerator += -neg_w * deviations[i_prime][m]
            denominator += abs(neg_w)
        except KeyError:
            # neighbor may not have rated the same movie
            # dont want to do dict lookup twice
            # so throw excetion
            pass

    if denominator == 0:
        prediction = averages[i]
    else:
        prediction = averages[i] + numerator / denominator
    prediction = min(5, prediction)   # max rating 5.0
    prediction = max(0.5, prediction) # min rating 0.5
    return prediction



In [ ]:
train_predictions = []
train_targets = []
for (i, m), target in usermovie2rating.items():
    # calc prediciton for this movie
    prediction = predict(i, m)
    train_predictions.append(prediction)
    train_targets.append(target)
    
test_predictions = []
test_targets = []
for (i, m), target in usermovie2rating_test.items():
    # calc prediciton for this movie
    prediction = predict(i, m)
    test_predictions.append(prediction)
    test_targets.append(target)

def mse(p, t):
    p = np.array(p)
    t = np.array(t)
    return np.mean((p-t)**2)


print('train mse:', mse(train_predictions, train_targets))
print('test mse:', mse(test_predictions, test_targets))

Movielens benchmark: a root mean squared error of about 0.8 or 0.9.

Now, you might think that our simple technique has done extremely well, but of course this is only a subset of the full amount of data.

So we've looked at about 150,000 reviews, if I remember correctly.

In any case, we can be pretty confident that user user collaborative filtering does pretty well.